# Model Training

## Notebook Setup

Import Libraries

In [3]:
# --- Standard ---
import os, json, math, time
from collections import Counter

# --- Numerical / data ---
import numpy as np
import pandas as pd

# --- Imaging & plotting ---
from PIL import Image

# --- Torch / ML ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# --- Metrics ---
from sklearn.metrics import f1_score, accuracy_score

# --- Vision ---
from torchvision import transforms

# --- Pretrained models ---
import timm

# --- Google Drive ---
from google.colab import drive

Set Training Data Path

In [4]:
drive.mount('/content/drive')
TRAIN_PATH = "/content/drive/MyDrive/neuro-imaging/data/Training"

Mounted at /content/drive


In [5]:
USE_SAVED_SPLITS = False
SPLITS_DIR = "/content/drive/MyDrive/neuro-imaging/splits"

## Path and Constants

In [6]:
TRAIN_PATH = "/content/drive/MyDrive/neuro-imaging/data/Training"
SPLITS_DIR = "/content/drive/MyDrive/neuro-imaging/splits"
OUT_DIR    = "/content/drive/MyDrive/neuro-imaging/models"
os.makedirs(OUT_DIR, exist_ok=True)

IMG_SIZE = 224
BATCH_SIZE = 32
LR = 3e-4
EPOCHS = 10
PATIENCE = 2
SEED = 42

## Reproducing

In [7]:
def set_seed(seed=SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


## Load Splits and Class Maps

In [8]:
def _read_paths(txt_path):
    with open(txt_path, "r") as f:
        return [ln.strip() for ln in f if ln.strip()]

with open(os.path.join(SPLITS_DIR, "class_to_idx.json"), "r") as f:
    class_to_idx = json.load(f)
idx_to_class = {v:k for k,v in class_to_idx.items()}
num_classes = len(class_to_idx)
print("Classes:", [idx_to_class[i] for i in range(num_classes)])

train_paths = _read_paths(os.path.join(SPLITS_DIR, "train.txt"))
val_paths   = _read_paths(os.path.join(SPLITS_DIR, "val.txt"))
print(f"Loaded {len(train_paths)} train, {len(val_paths)} val files")


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
Loaded 4855 train, 857 val files


## Transforms

In [9]:
train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

## Dataset from path list

In [10]:
class PathDataset(Dataset):
    def __init__(self, paths, class_to_idx, transform=None):
        self.paths = paths
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, i):
        p = self.paths[i]
        cls_name = os.path.basename(os.path.dirname(p))
        y = self.class_to_idx[cls_name]
        img = Image.open(p)
        if self.transform:
            img = self.transform(img)
        return img, y

train_ds = PathDataset(train_paths, class_to_idx, train_tfms)
val_ds   = PathDataset(val_paths,   class_to_idx, val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

## Class weights (handle imbalances)

In [11]:
train_counts = Counter([os.path.basename(os.path.dirname(p)) for p in train_paths])
counts_by_idx = np.array([train_counts[idx_to_class[i]] for i in range(num_classes)], dtype=np.float32)
weights = (len(train_paths) / (num_classes * counts_by_idx))
class_weights = torch.tensor(weights, dtype=torch.float32, device=device)
print("Class weights:", weights.tolist())

Class weights: [1.080810308456421, 1.0665642023086548, 0.8950958847999573, 0.980411946773529]


## Model, loss, optimizer, scaler

In [12]:
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=LR)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

/tmp/ipython-input-3194161557.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))


## Train / eval helpers

In [13]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    all_preds, all_targets = [], []
    total_loss = 0.0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    val_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(all_targets, all_preds)
    f1_macro = f1_score(all_targets, all_preds, average="macro")
    return {"loss": val_loss, "acc": acc, "f1_macro": f1_macro}

def train_one_epoch(model, loader, device):
    model.train()
    total_loss = 0.0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

## Training loop with early stopping on val macro-F1

In [16]:
best = {"f1": -1.0, "epoch": -1}
history = []
patience_left = PATIENCE  # initialize once

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()
    train_loss = train_one_epoch(model, train_dl, device)
    val_metrics = evaluate(model, val_dl, device)
    elapsed = time.time() - t0

    log = {
        "epoch": epoch,
        "train_loss": float(train_loss),
        "val_loss": float(val_metrics["loss"]),
        "val_acc": float(val_metrics["acc"]),
        "val_f1_macro": float(val_metrics["f1_macro"]),
        "time_sec": float(elapsed),
        "lr": float(optimizer.param_groups[0]["lr"]),
    }
    history.append(log)
    print(log)

    # Early stopping on macro-F1
    if val_metrics["f1_macro"] > best["f1"] + 1e-6:
        best.update({"f1": val_metrics["f1_macro"], "epoch": epoch})
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "class_to_idx": class_to_idx,
                "img_size": IMG_SIZE,
            },
            os.path.join(OUT_DIR, "efficientnet_b0_best.pt"),
        )
        patience_left = PATIENCE  # reset patience
    else:
        patience_left -= 1

    if patience_left <= 0:
        print(f"Early stopping at epoch {epoch} "
              f"(best epoch {best['epoch']}, F1={best['f1']:.4f})")
        break

# Save training log
with open(os.path.join(OUT_DIR, "train_log.json"), "w") as f:
    json.dump(history, f, indent=2)

print("Best:", best)

/tmp/ipython-input-2394693305.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


KeyboardInterrupt: 